oft wird rag gebaut mit pdfs, und wenn man so ein system aufbaut kann man sicherlich eine grössere user base ansprechen, welche das system dann auch bauen möchten. in diesem projekt möchte ich aber webscraping von yahoo finance machen, weil viele leute auch yahoo finance lesen und auf yahoo finance viel mehr informationen sind. ich kann dort webscraping machen von bestimmten aktien, fonds oder indexes und dann alle informationen scrapen von kursen, artikeln, stock prices,
intraday trading data,
option prices;
company disclosures and information such as :
financial information,
earnings,
analyst ratings and recommendations,
insider trades,
SEC filings,
news,
large shareholders;

 dann kann ich all das in einen ordner speichern und einen rag bauen auf all diese sachen. das spart einerseits zeit weil man nicht manuell nach pdfs online suchen muss zu den gewünschten themen, sondern man kann alles zeitsparend an einem ort scrapen und dann weiterbenutzen als input für das rag.

 denn technisch gesehen macht LangChain keinen Unterschied, ob deine Daten ursprünglich aus PDFs, TXTs, Webseiten oder JSON stammen. Wichtig ist nur, was in den vektorstore eingespeist wird, ist reiner text, nicht das ursprüngliche Dateiformat (wie z.b. pdf).

 Ob du folgenden Text aus einem PDF extrahierst:
 In 2023, Allianz reported a revenue of €152.7 billion and a net income of €14.2 billion.

oder denselben Text aus einem JSON-Feld zusammenstellst wie:
{ "revenue": "€152.7 billion", "net_income": "€14.2 billion" }

ist für den Vektorstore und LangChain völlig egal, solange du am Ende diesen Text als Chunk übergibst.

📘 Warum sieht man online meist PDFs?

PDFs sind realistische Quellen (z. B. Geschäftsberichte, Studien)
Sie demonstrieren komplexere Verarbeitung (PDF → Text → Chunk → Embedding)
Viele Nutzer haben genau solche Dateien → Tutorials wollen für die breite Masse relevant sein
JSON ist oft maschinenlesbar, aber in der Praxis weniger "typisch" für Endnutzer
✅ Wann ist JSON sogar besser?

Wenn du strukturierte, tabellarische Daten hast (Finanzdaten, ESG-KPIs, Zeitreihen)
Wenn du Daten aus APIs sammelst (z. B. von Morningstar, AlphaVantage, Unternehmensdatenbanken)
Wenn du dein System voll automatisiert aufbaust ohne manuelles PDF-Verarbeiten
Wenn du bestimmte Felder gezielt extrahieren und in Textform bringen willst

Ja, du kannst problemlos ein LangChain-basiertes RAG-System mit JSON-Dateien aufbauen.
🧱 Die einzige Voraussetzung: Du musst den Inhalt in Klartext-Chunks bringen, damit das LLM damit arbeiten kann.



# source .venv/bin/activate 

In [ ]:
import yfinance as yf
import pandas
import matplotlib
from datetime import datetime
from pytz import timezone


In [ ]:
# stock price download
# by default hat man nur daily prices
# all prices are adjusted for dividends and splits

yf.download(['AAPL', 'NVDA'])

[*********************100%***********************]  2 of 2 completed


Price            Close                    High                     Low  \
Ticker            AAPL        NVDA        AAPL        NVDA        AAPL   
Date                                                                     
1980-12-12    0.098726         NaN    0.099155         NaN    0.098726   
1980-12-15    0.093575         NaN    0.094005         NaN    0.093575   
1980-12-16    0.086707         NaN    0.087136         NaN    0.086707   
1980-12-17    0.088853         NaN    0.089282         NaN    0.088853   
1980-12-18    0.091429         NaN    0.091859         NaN    0.091429   
...                ...         ...         ...         ...         ...   
2025-04-08  172.419998   96.300003  190.339996  105.849998  169.210007   
2025-04-09  198.850006  114.330002  200.610001  115.099998  171.889999   
2025-04-10  190.419998  107.570000  194.779999  110.860001  183.000000   
2025-04-11  198.149994  110.930000  199.539993  111.550003  186.059998   
2025-04-14  202.369995  110.370003  212.940002  114.290001  201.162094   

Price                         Open                 Volume               
Ticker            NVDA        AAPL        NVDA       AAPL         NVDA  
Date                                                                    
1980-12-12         NaN    0.098726         NaN  469033600          NaN  
1980-12-15         NaN    0.094005         NaN  175884800          NaN  
1980-12-16         NaN    0.087136         NaN  105728000          NaN  
1980-12-17         NaN    0.088853         NaN   86441600          NaN  
1980-12-18         NaN    0.091429         NaN   73449600          NaN  
...                ...         ...         ...        ...          ...  
2025-04-08   94.459999  186.699997  103.809998  120859500  476243400.0  
2025-04-09   97.529999  171.949997   98.889999  184395900  612918300.0  
2025-04-10   99.150002  189.070007  109.370003  121880000  437812400.0  
2025-04-11  107.480003  186.100006  108.500000   87300000  312853800.0  
2025-04-14  109.070000  211.440002  114.139999   65385598  167767788.0  

[11175 rows x 10 columns]

In [ ]:
# data for a specific stock 
# dictionary of meta data and information about a specific stock

aapl = yf.Ticker('AAPL')

aapl.info

{'address1': 'One Apple Park Way',
 'city': 'Cupertino',
 'state': 'CA',
 'zip': '95014',
 'country': 'United States',
 'phone': '(408) 996-1010',
 'website': 'https://www.apple.com',
 'industry': 'Consumer Electronics',
 'industryKey': 'consumer-electronics',
 'industryDisp': 'Consumer Electronics',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and p

In [18]:
aapl.news

[{'id': '346e3739-8006-4bf4-be1d-4997396dc773',
  'content': {'id': '346e3739-8006-4bf4-be1d-4997396dc773',
   'contentType': 'STORY',
   'title': "'Trump does Tim Apple a solid': Investors cheer smartphone tariff exemption despite mixed White House signals",
   'description': '',
   'summary': 'President Trump stoked confusion this past weekend about tariffs on smartphones, but investors are viewing an electronics exemption announced Friday night as an important temporary win for Apple and other China-dependent technology giants.',
   'pubDate': '2025-04-14T12:42:35Z',
   'displayTime': '2025-04-14T16:40:36Z',
   'isHosted': True,
   'bypassModal': False,
   'previewUrl': None,
   'thumbnail': {'originalUrl': 'https://s.yimg.com/os/creatr-uploaded-images/2025-02/996ee540-f24d-11ef-bf5f-be1985ce6685',
    'originalWidth': 5118,
    'originalHeight': 3414,
    'caption': '',
    'resolutions': [{'url': 'https://s.yimg.com/uu/api/res/1.2/ZlGdQJTJlY8KjzjxQ4tEvA--~B/aD0zNDE0O3c9NTExODthcHB

In [19]:
aapl.earnings_dates

,EPS Estimate,Reported EPS,Surprise(%)
Earnings Date,,,
2026-01-28 16:00:00-05:00,NaN,NaN,NaN
2025-10-29 06:59:00-04:00,NaN,NaN,NaN
2025-07-30 06:59:00-04:00,NaN,NaN,NaN
2025-05-01 16:00:00-04:00,NaN,NaN,NaN
2025-01-30 16:31:00-05:00,2.35,2.40,2.15
2024-10-31 16:31:00-04:00,1.60,1.64,2.35
2024-08-01 16:30:00-04:00,1.35,1.40,3.99
2024-05-02 16:31:00-04:00,1.50,1.53,1.97
2024-02-01 16:00:00-05:00,2.10,2.18,3.90


In [20]:
aapl.insider_transactions

,Shares,Value,URL,Text,Insider,Position,Transaction,Start Date,Ownership
0,108136,24184658.0,,Sale at price 221.77 - 224.76 per share.,COOK TIMOTHY D,Chief Executive Officer,,2025-04-02,D
1,35493,7950691.0,,Sale at price 223.48 - 225.03 per share.,WILLIAMS JEFFREY E,Chief Operating Officer,,2025-04-02,D
2,38822,8683252.0,,Sale at price 221.68 - 224.62 per share.,ADAMS KATHERINE L,General Counsel,,2025-04-02,D
3,218568,NaN,,,COOK TIMOTHY D,Chief Executive Officer,,2025-04-01,D
4,74535,NaN,,,O'BRIEN DEIRDRE,Officer,,2025-04-01,D
...,...,...,...,...,...,...,...,...,...
73,15419,2753217.0,,Sale at price 178.56 per share.,O'BRIEN DEIRDRE,Officer,,2023-08-07,D
74,31896,NaN,,,O'BRIEN DEIRDRE,Officer,,2023-08-04,D
75,68642,0.0,,Stock Gift at price 0.00 per share.,JUNG ANDREA,Director,,2023-05-16,I
76,2900,0.0,,Stock Gift at price 0.00 per share.,ADAMS KATHERINE L,General Counsel,,2023-05-10,D


In [21]:
aapl.institutional_holders

,Date Reported,Holder,pctHeld,Shares,Value,pctChange
0,2024-12-31,Vanguard Group Inc,0.0929,1395785512,288271580089,0.0365
1,2024-12-31,Blackrock Inc.,0.0748,1123417607,232019437002,0.0272
2,2024-12-31,State Street Corporation,0.0396,595501507,122988925513,0.0197
3,2024-12-31,"FMR, LLC",0.0227,341640974,70559109943,-0.0611
4,2024-12-31,"Geode Capital Management, LLC",0.0226,340164913,70254259066,0.0189
5,2024-12-31,"Berkshire Hathaway, Inc",0.0200,300000000,61958999633,0.0000
6,2024-12-31,Morgan Stanley,0.0159,238264141,49208692749,0.0294
7,2024-12-31,Price (T.Rowe) Associates Inc,0.0147,220107535,45458808934,-0.0657
8,2024-12-31,NORGES BANK,0.0125,187160162,38654188029,0.0542
9,2024-12-31,JPMORGAN CHASE & CO,0.0122,183008151,37796673202,0.0206


In [22]:
aapl.major_holders

Breakdown,Value
insidersPercentHeld,0.02070
institutionsPercentHeld,0.62829
institutionsFloatPercentHeld,0.64158
institutionsCount,6960.00000


In [23]:
aapl.mutualfund_holders

,Date Reported,Holder,pctHeld,Shares,Value,pctChange
0,2024-12-31,VANGUARD INDEX FUNDS-Vanguard Total Stock Mark...,0.0315,473591874,97810929159,0.0344
1,2024-12-31,VANGUARD INDEX FUNDS-Vanguard 500 Index Fund,0.0272,409170316,84505944864,0.0279
2,2025-02-28,Fidelity Concord Street Trust-Fidelity 500 Ind...,0.0126,189316298,39099494794,0.0045
3,2025-02-28,SPDR S&P 500 ETF TRUST,0.0124,186864109,38593044203,0.0138
4,2025-02-28,iShares Trust-iShares Core S&P 500 ETF,0.0118,177551321,36669674109,0.0016
5,2024-12-31,VANGUARD INDEX FUNDS-Vanguard Growth Index Fund,0.0101,152087687,31410669810,0.1240
6,2025-03-31,"Invesco QQQ Trust, Series 1",0.0084,125730621,25967145001,-0.0035
7,2024-12-31,VANGUARD INSTITUTIONAL INDEX FUNDS-Vanguard In...,0.0064,95471977,19717827293,-0.0127
8,2024-11-30,VANGUARD WORLD FUND-Vanguard Information Techn...,0.0045,66889141,13814614209,-0.0577
9,2025-02-28,iShares Trust-iShares Russell 1000 Growth ETF,0.0034,50848704,10501782775,0.0006


In [24]:
aapl.sec_filings

[{'date': datetime.date(2025, 4, 4),
  'epochDate': 1743724800,
  'type': 'SD',
  'title': 'Specialized Disclosure Report filed pursuant to Section 1502 of the Dodd-Frank Wall Street Reform and Consumer Protection Act relating to the use of conflict minerals (Rule 13p-1)',
  'edgarUrl': 'https://finance.yahoo.com/sec-filing/AAPL/0001140361-25-012359_320193',
  'exhibits': {'SD': 'https://cdn.yahoofinance.com/prod/sec-filings/0000320193/000114036125012359/ef20046400_sd.htm',
   'EX-1.01': 'https://cdn.yahoofinance.com/prod/sec-filings/0000320193/000114036125012359/ef20046400_ex1-01.htm'},
  'maxAge': 1},
 {'date': datetime.date(2025, 2, 25),
  'epochDate': 1740441600,
  'type': '8-K',
  'title': 'Corporate Changes & Voting Matters',
  'edgarUrl': 'https://finance.yahoo.com/sec-filing/AAPL/0001140361-25-005876_320193',
  'exhibits': {'EXCEL': 'https://s3.amazonaws.com/finance-pri-uw2/sec-filings/0000320193/000114036125005876/Financial_Report.xlsx',
   '8-K': 'https://cdn.yahoofinance.com

In [25]:
ticker = yf.Ticker("AAPL")
info = ticker.info  # oder .get_info() je nach Version

print(info["longName"])
print(info["sector"])
print(info["industry"])
print(info["fullTimeEmployees"])
print(info["website"])

Apple Inc.
Technology
Consumer Electronics
150000
https://www.apple.com


In [28]:
print(info["trailingPE"])        # KGV
print(info["forwardPE"])         # zukünftiger KGV
print(info["dividendYield"])     # Dividendenrendite
print(info["marketCap"])         # Marktkapitalisierung
print(info["totalRevenue"])      # Umsatz
print(info["grossProfits"])      # Bruttogewinn


32.661903
24.761732
0.5
3091097714688
395760009216
184102993920


In [34]:
financials = ticker.financials
print(financials)


                                                        2024-09-30  \
Tax Effect Of Unusual Items                                    0.0   
Tax Rate For Calcs                                           0.241   
Normalized EBITDA                                   134661000000.0   
Net Income From Continuing Operation Net Minori...   93736000000.0   
Reconciled Depreciation                              11445000000.0   
Reconciled Cost Of Revenue                          210352000000.0   
EBITDA                                              134661000000.0   
EBIT                                                123216000000.0   
Net Interest Income                                            NaN   
Interest Expense                                               NaN   
Interest Income                                                NaN   
Normalized Income                                    93736000000.0   
Net Income From Continuing And Discontinued Ope...   93736000000.0   
Total Expenses      

In [35]:
# balance sheet
balance = ticker.balance_sheet
print(balance)


                                                      2024-09-30  \
Treasury Shares Number                                       NaN   
Ordinary Shares Number                             15116786000.0   
Share Issued                                       15116786000.0   
Net Debt                                           76686000000.0   
Total Debt                                        106629000000.0   
...                                                          ...   
Cash Cash Equivalents And Short Term Investments   65171000000.0   
Other Short Term Investments                       35228000000.0   
Cash And Cash Equivalents                          29943000000.0   
Cash Equivalents                                    2744000000.0   
Cash Financial                                     27199000000.0   

                                                      2023-09-30  \
Treasury Shares Number                                       0.0   
Ordinary Shares Number                         

In [36]:
cashflow = ticker.cashflow
print(cashflow)


                                                    2024-09-30  \
Free Cash Flow                                  108807000000.0   
Repurchase Of Capital Stock                     -94949000000.0   
Repayment Of Debt                                -9958000000.0   
Issuance Of Debt                                           0.0   
Issuance Of Capital Stock                                  NaN   
Capital Expenditure                              -9447000000.0   
Interest Paid Supplemental Data                            NaN   
Income Tax Paid Supplemental Data                26102000000.0   
End Cash Position                                29943000000.0   
Beginning Cash Position                          30737000000.0   
Changes In Cash                                   -794000000.0   
Financing Cash Flow                            -121983000000.0   
Cash Flow From Continuing Financing Activities -121983000000.0   
Net Other Financing Charges                      -5802000000.0   
Cash Divid

In [37]:
# Finanzdaten von SEC Edgar

from edgar import Company, set_identity


In [38]:
set_identity("Moreno Gallo gallomor@students.zhaw.ch")

In [40]:
apple = Company("AAPL")
apple

╭────────────────────────────────────────── 🏢 Apple Inc. [320193] AAPL ──────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│ ╭───────────────────────────────────────────────── 📋 Entity ─────────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ │   CIK   320193   Type   Operating   ○                                                                       │ │
│ │                                                                                                             │ │
│ │                                                                                                             │ │
│ │   Category                  Industry                     Fiscal Year E

In [42]:
apple.former_names

[{'name': 'APPLE INC', 'from': '2007-01-10', 'to': '2019-08-05'},
 {'name': 'APPLE COMPUTER INC', 'from': '1994-01-26', 'to': '2007-01-04'},
 {'name': 'APPLE COMPUTER INC/ FA', 'from': '1997-07-28', 'to': '1997-07-28'}]

In [45]:
filings = apple.get_filings(form="10-Q")
filings.to_pandas

<bound method Filings.to_pandas of ╭──────────────────────────────────────── Filings for Apple Inc. [320193] ────────────────────────────────────────╮
│                                                                                                                 │
│    #   Form       Description                                          Filing Date   Accession Number           │
│  ─────────────────────────────────────────────────────────────────────────────────────────────────────────      │
│    0   10-Q       Quarterly report for public companies                2025-01-31    0000320193-25-000008       │
│    1   10-Q       Quarterly report for public companies                2024-08-02    0000320193-24-000081       │
│    2   10-Q       Quarterly report for public companies                2024-05-03    0000320193-24-000069       │
│    3   10-Q       Quarterly report for public companies                2024-02-02    0000320193-24-000006       │
│    4   10-Q       Quarterly report 